# TM10007 Assignment template

In [28]:
# Run this to use from colab environment
!pip install -q --upgrade git+https://github.com/karinvangarderen/tm10007_project.git

## Data loading and cleaning

Below are functions to load the dataset of your choice. After that, it is all up to you to create and evaluate a classification method. Beware, there may be missing values in these datasets. Good luck!

In [0]:
# Importing modules
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import LabelEncoder
from sklearn.preprocessing import StandardScaler
import numpy as np
import pandas as pd
import math
from sklearn.decomposition import PCA

In [34]:
# Data loading functions. Uncomment the one you want to use
# from adni.load_data import load_data
from brats.load_data import load_data
#from hn.load_data import load_data
# hoi
# en als ik dit type?

data = load_data()
print(f'The number of samples: {len(data.index)}')
print(f'The number of columns: {len(data.columns)}')


The number of samples: 167
The number of columns: 725


In [0]:
data_top = data.head()  

# display  
# data_top # = show data_top



---


# Outline opdracht


---


Gebaseerd op beeldvorming moeten we eruit kunnen halen wat voor een soort tumor het is?
Je wil weten of het HIGH of LOW grade is.

In [36]:
# Preprocessing the data 

# Drop columns which contain NaN values
threshold = math.floor(len(data)/2)  # calculate threshold, half of total rows
data_drop = data.dropna(thresh=threshold, axis=1)  # Delete columns/features with more than 'threshold' NaNs
data_drop = data_drop.dropna(axis=0)  # Delete rows/subjects with NaNs
print(data_drop)

# Split data and labels
labels = data_drop['label']
data_drop = data_drop.drop(columns="label")  # Data without labels

# Convert labels 'GBM' and 'LGG' to respectively 0 and 1
label_encoder = LabelEncoder()
labels = label_encoder.fit_transform(labels)

# Delete columns with strings (2 columns)
# data_strings = data_drop.select_dtypes(include=[object])
# columns_strings = list(data_strings.columns)
# data_no_strings = data_drop.drop(columns_strings, axis=1)

# Replace infinity values with NaN
#data_no_strings = data_no_strings.replace([np.inf, -np.inf], np.nan)
# print(np.isinf(data_no_strings.values).any()) - check if dataframe contains infinity values

# Feature scaling
standard_scaler = StandardScaler()
data_scaled = standard_scaler.fit_transform(data_drop.values)
data_df = pd.DataFrame(data_scaled, index = data_drop.index, columns = data_drop.columns)

# Split the data in a train (80%) and test set (20%)
data_train, data_test, label_train, label_test = train_test_split(data_df, labels, test_size=0.2)

# PCA
  # Training set
pca_train = PCA(n_components=20)  # Create a PCA with 20 components
pca_train.fit(data_train)  # Fit PCA
data_train_pca = pca_train.transform(data_train)  # Transform train data using PCA
df_train_pca = pd.DataFrame(data_train_pca, index = data_train.index)  # Put train data back in dataframe with 20 most important features
  # Test set
data_test_pca = pca_train.transform(data_test)  # Transform test data using PCA


              VOLUME_ET  VOLUME_NET  VOLUME_ED  ...  TGM_Cog_Z_1    TGM_T_1  label
ID                                              ...                               
TCGA-02-0006       1662         384      36268  ...     0.052741    2.00000    GBM
TCGA-02-0009       4362        4349      15723  ...     0.094336   91.47360    GBM
TCGA-02-0011      33404       48612      45798  ...     0.096035  272.42900    GBM
TCGA-02-0027      12114        7587      34086  ...     0.096470  128.46800    GBM
TCGA-02-0033      34538        7137      65653  ...     0.096894  240.77800    GBM
...                 ...         ...        ...  ...          ...        ...    ...
TCGA-HT-7694       1036      189152     171595  ...     0.090456  719.23800    LGG
TCGA-HT-8018       2093        8685      39142  ...     0.054307    2.00000    LGG
TCGA-HT-8111       1929         437      54079  ...     0.126712    7.06744    LGG
TCGA-HT-8114       8755      168606      11325  ...     0.107433   15.52240    LGG
TCGA

Vanaf hier begin van SVM. 

In [37]:
!pip install SimpleITK 
# General packages
import numpy as np 
import matplotlib.pyplot as plt
from sklearn import datasets as ds
import SimpleITK as sitk

# Classifiers and kernels
from sklearn.neighbors import KNeighborsClassifier
from sklearn.ensemble import RandomForestClassifier
from sklearn.svm import SVC
from sklearn.decomposition import PCA, KernelPCA
from sklearn.kernel_approximation import RBFSampler
from sklearn.metrics.pairwise import rbf_kernel, sigmoid_kernel

In [48]:
# Construct classifiers
svmlin = SVC(kernel='linear', gamma='scale')  # linear kernel
svmrbf = SVC(kernel='rbf', gamma='scale')  # radial basis function kernel
svmpoly = SVC(kernel='poly', degree=3, gamma='scale')  # polynomial kernel

clsfs = [svmlin, svmpoly, svmrbf] # verschillende classifiers in lijst
for clf in clsfs: # voor alle classifiers in lijst
   clf.fit(data_train_pca, label_train) # fit classifier op trainingsset
   test = clf.predict(data_test_pca) # test classifier op testset
   error = sum(abs(test - label_test)) # hoevaak komt het niet overeen met gegeven label?
   print(f'Misclassified: {error}/{len(test)}')




Misclassified: 6/30
Misclassified: 5/30
Misclassified: 4/30
